In [ ]:
import urllib.request
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal

In [ ]:
plt.style.use("seaborn-talk")

In [ ]:
project = Path.cwd()
output = project / "output"
site_info_output = output / "site_info"
discharge_data = output / "discharge_data"
delft3d_input = output / "delft3d_input"
input_data = project / "input"
figures = project / "figures"

In [ ]:
# some constants
cfs2cms = 0.0283168

# functions

In [ ]:
def apply_butterworth(discharge, buff=20, dts=25, N=5):
    """apply butterworth filter to remove tidal influence from data
    
    input:
    discharge = discharge dataframe
    dts       = sampling interval in minutes
    N         = filter order
    
    returns:
    filtered dataframe
    
    """
    # parameters
    crit_freq = 1/(((24.8412 + buff)*60*60))  # lundar day in hours to Hz
    fs = 1/(dts*60)  # sampling frequency
    
    b, a = signal.butter(N, crit_freq, btype='lowpass', fs=fs)

    filtered = discharge.apply(lambda x: signal.filtfilt(b, a, x))
    filtered.columns = ['discharge_cms_Butterworth_filtered']
    
    return filtered

def apply_godin(discharge):
    """apply Godin filter to remove tidal influence from data
    
    input:
    discharge = discharge dataframe
    
    returns:
    filtered dataframe
    
    """
    # parameters
    # Godin filter (USGS standard)
    godin = discharge.resample('1H').mean().interpolate(method='time').rolling(
        window=24, center=True).mean().rolling(
        window=25, center=True).mean().rolling(
        window=25, center=True).mean()
    godin.columns = ['discharge_cms_Godin_filtered']
    
    return godin

def combine_discharges(discharges):
    """combine discharge data after interpolating to common index
    
    input = list of discharge dataframes
    
    returns combined discharge dataframe
    """
    dedups = []
    regular_data = []
    for discharge in discharges:
        dedups.append(discharge[~discharge.index.duplicated()])
   
    new_index = dedups[0].index
    for dedup in dedups[1:]:
        new_index = new_index.union(dedup.index).unique()
   
    for dedup in dedups:
        regular_data.append(dedup.reindex(new_index).interpolate(method="linear"))
        pass
    
    discharge_combined = regular_data[0]
    for regs in regular_data[1:]:
        discharge_combined += regs
    
    return discharge_combined

def download_discharge_nwis(
    site_name, site_no, begin_date, end_date, data_code=60, skiprows=28
):
    """download data from https://nwis.waterdata.usge and outputs as dataframe

    inputs:
    site_name = user specified name for site
    site_no = USGS site number code
    begin_date = first day in timeseries (YYYY-MM-dd)
    end_date = last day in timeseries (YYYY-MM-dd)
    skiprows = number of header rows to skip (default=28)

    return = discharge (pandas DataFrame)
    """

    # some constants
    cfs2cms = 0.0283168

    # output file and request
    out_fn = discharge_data / f"{site_name}_{site_no}_{begin_date}_{end_date}.txt"
    request = f"https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_{data_code:05d}=on&format=rdb&site_no={site_no}&period=&begin_date={begin_date}&end_date={end_date}"

    # get data
    txt, http = urllib.request.urlretrieve(request, out_fn)

    # Pandas
    try:
        discharge = pd.read_csv(
            txt,
            sep="\s+",
            skiprows=skiprows,
            usecols=[2, 3, 5],
            parse_dates={"datetime_CST": [0, 1]},
            header=0,
            index_col=0,
            names=["date", "time", "discharge"],
            dtype={"discharge": float}
        )
    except:
        print("Problem with parsing text ")
        raise

    try:
        discharge.discharge = discharge.discharge * cfs2cms
    except TypeError as e:
        print("There is a problem with DataFrame structure/dtypes")
        raise

    try:
        discharge.index = (
            discharge.index.tz_localize("America/Chicago", ambiguous=True)
            .tz_convert("UTC")
            .tz_localize(None)
        )
    except AttributeError as e:
        print("Problem converting datetime to UTC. Check data")
        raise

    discharge.to_csv(
        discharge_data / f"{site_name}_{begin_date}.csv",
        sep="\t",
        header=["discharge_m3s"],
        index_label=["datetime_UTC"],
    )
    return discharge


def download_mean_discharge_nwis(
    site_name, site_no, begin_date, end_date, data_code=60, skiprows=28
):
    """download data from https://nwis.waterdata.usge and outputs as dataframe

    inputs:
    site_name = user specified name for site
    site_no = USGS site number code
    begin_date = first day in timeseries (YYYY-MM-dd)
    end_date = last day in timeseries (YYYY-MM-dd)
    skiprows = number of header rows to skip (default=28)

    return = discharge (pandas DataFrame)
    """

    # some constants
    cfs2cms = 0.0283168

    # output file and request
    out_fn = discharge_data / f"{site_name}_{site_no}_{begin_date}_{end_date}.txt"
    request = f"https://nwis.waterdata.usgs.gov/nwis/dv?cb_{data_code:05d}=on&format=rdb&referred_module=sw&site_no={site_no}&period=&begin_date={begin_date}&end_date={end_date}"

    # get data
    txt, http = urllib.request.urlretrieve(request, out_fn)

    # Pandas
    try:
        discharge = pd.read_csv(
            txt,
            sep="\s+",
            skiprows=skiprows,
            usecols=[2, 3],
            parse_dates={"datetime_CST": [0]},
            header=0,
            index_col=0,
            names=["date", "discharge"],
        )
    except:
        print("Problem with parsing text data")
        raise

    try:
        discharge = discharge * cfs2cms
    except TypeError as e:
        print("There is a problem with DataFrame structure/dtypes")
        raise

    try:
        discharge.index = (
            discharge.index.tz_localize("America/Chicago", ambiguous=True)
            .tz_convert("UTC")
            .tz_localize(None)
        )
    except AttributeError as e:
        print("Problem converting datetime to UTC. Check data")
        raise

    discharge.to_csv(
        discharge_data / f"{site_name}_{begin_date}.csv",
        sep="\t",
        header=["discharge_m3s"],
        index_label=["datetime_UTC"],
    )
    return discharge

# Bonnet Carre Spillway

In [ ]:
# calculate Bonnet Carre Spillway (BCS) discharge
# opening data form MVD
data_fn = input_data / "bonnet_carre_2016-2019.csv"
opening = pd.read_csv(data_fn, parse_dates=True, index_col=0, usecols=[0, 2])

# days in model 
not_open_idx = pd.date_range(start="2015-12-01", end="2020-01-02", freq="D")
not_open = pd.DataFrame(
    data=np.zeros(len(not_open_idx)),
    index=not_open_idx,
    columns=["discharge_cms"],
)

leakage = 7500 * cfs2cms

discharge = opening + not_open
discharge = discharge.where(discharge > leakage, leakage)
discharge = discharge.where(~discharge.isna(), leakage)

# convert time stamp to UTC
discharge.index = (
    discharge.index.tz_localize("America/Chicago", ambiguous=True)
    .tz_convert("UTC")
    .tz_localize(None)
)
discharge.to_csv(
    delft3d_input / "Bonnet_Carre_2016-2020_converted.csv",
    sep="\t",
    header=["discharge_cms"],
    index_label=["datetime_UTC"],
)

# visualize
discharge.plot(figsize=(12, 5))
plt.ylabel("discharge [$m^3/s$]")
plt.title("Bonnet Carré Spillway")
plt.savefig(figures / "bonnet_carre_2016-2020.png", bbox_inches="tight")

# Site descriptions

In [ ]:
# potential sites to be used
sites = {
    "jourdan_river": ("02481660", "St. Louis Bay"),  # St. Louise Bay
    "tuxachanie_creek": (
        "02480500",
        "Biloxi Bay",
    ),  # Biloxi Bay (need to extrapolate form old stats)
    "tchoutacabouffa_river": ("02480345", "Biloxi Bay"),  # Biloxi Bay
    "pascagoula_river": ("02479310", "Pascagoula River"),  # Pascagoula River
    "escatawpa_river_at_I-10": ("0248018020", "Pascagoula River"),  # Pascagoula River
    "escatawpa_river_nr_agricola": ("02479560", "Pascagoula River"),  # Pascagoula River
    "atchafalaya_at_morgan_city": ("07381600", "Atchafalaya River"),  # Atchafalaya
    "wax_lake_outlet_at_calumet": ("07381590", "Wax Lake"),  # Wax Lake
    "mississippi_at_belle_chase": ("07374525", "Mississippi River"),  # MS River
    "new_canal_river_east_sorrento": (
        "073802284",
        "Blind River",
    ),  # Used for Blind River
    "amite_river_at_port_vincent": ("07380120", "Amite River"),  # Used for Amite
    "tickfaw_river_at_holden": ("07376000", "Tickfaw River"),  # Used for Tickfaw
    "natalbany_river_at_baptist": ("07376500", "Tickfaw River"),  # Used for Tickfaw
    "tangipahoa_river_at_robert": (
        "07375500",
        "Tangipahoa River",
    ),  # Used for Tangipahoa
    "tchefuncte_near_folsom": ("07375000", "Tchefuncte River"),  # Used for Tchefuncte
    "bogue_chitto_river": ("02492000", "Pearl River"),  # Used for pearl
    "pearl_at_bogalusa": ("02489500", "Pearl River"),  # Used for pearl
    "pearl_at_walkiah": ("02492110", "Pearl River"),  # Used for pearl
    "pearl_at_nstl_station": ("02492620", "Pearl River"),  # Used for pearl
    "pearl_at_cxs_railroad": ("301141089320300", "Pearl River"),  # Used for pearl
    "west_hobolochitto": ("02492360", "Pearl River"),  # Used for pearl
    "east_hobolochitto": ("02492343", "Pearl River"),  # Used for pearl
    "wolf_river": ("02481510", "St. Louis Bay"),  # St. Louise Bay
    "biloxi_river": ("02481000", "Biloxi Bay"),  # Biloxi Bay
    "red_creek_at_vestry": ("02479300", "Pascagoula River"),  # Pascagoula River
    "alabama_river": ("02428400", "Head of Mobile Bay"),  # Head of Mobile Bay
    "mobile_river_at_bucks": ("02470629", "Head of Mobile Bay"),  # Head of Mobile Bay
    "tensaw_at_mount_vernon": ("02471019", "Head of Mobile Bay"),  # Head of Mobile Bay
    "tombigbee": ("02469761", "Head of Mobile Bay"),  # Head of Mobile Bay
    "chickasaw_creek": ("02471001", "Head of Mobile Bay"),  # Head of Mobile Bay
    "fowl_river": ("02471078", "Mouth of Mobile Bay"),  # Mouth of Mobile Bay
    "fish_river": ("02378500", "Mouth of Mobile Bay"),  # Mouth of Mobile Bay
    "magnolia_river": ("02378300", "Mouth of Mobile Bay"),  # Mouth of Mobile Bay
}

skiprows = 30
site_infos = []

for site_name, (site_no, bc_source) in sites.items():
    out_fn = site_info_output / f"{site_name}_{site_no}_site_info.txt"
    request = f"https://waterservices.usgs.gov/nwis/site/?format=rdb&sites={site_no}"

    if not out_fn.exists():

        try:
            txt, http = urllib.request.urlretrieve(request, out_fn)
        except:
            print(f"{site_name} has no data!!!")
            continue
        pass
    else:
        txt = out_fn
        pass

    site_info = pd.read_csv(
        txt,
        sep="\t",
        skiprows=skiprows,
        usecols=[1, 2, 4, 5],
        header=0,
        index_col=0,
        names=["site_id", "long_name", "lat", "lon"],
        dtype={"site_id": str},
    )
    site_info["site_name"] = site_name
    site_info["bc_source"] = bc_source
    site_info["used"] = False
    site_info2 = site_info.copy()
    site_infos.append(site_info)

site_info = pd.concat(site_infos)
site_info = gpd.GeoDataFrame(
    site_info,
    geometry=gpd.points_from_xy(site_info.lon, site_info.lat),
    crs="EPSG:4326",
)

# Download, filter, visualize and write

In [ ]:
# download
begin_date = "2015-12-01"
end_date = "2020-01-02"

## Atchafalaya River

In [ ]:
site_name = "atchafalaya_at_morgan_city"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_atchafalaya = download_discharge_nwis(site_name, site_no, begin_date, end_date, skiprows=30)
q_atchafalaya_godin = apply_godin(q_atchafalaya)

# visualize
fig, axes = plt.subplots(nrows=2, figsize=(16, 9))
q_atchafalaya.plot(ax=axes[0])
q_atchafalaya_godin.plot(ax=axes[0])
axes[0].set_ylabel('discharge [$m^3/s$]')
axes[0].set_title(site_name)

q_atchafalaya.plot(ax=axes[1])
q_atchafalaya_godin.plot(ax=axes[1])
axes[1].set_xlim(pd.to_datetime('2018-02-07'), pd.to_datetime('2018-04-29'))
axes[1].set_ylabel('discharge [$m^3/s$]')
fig.savefig(figures / f'{site_name}_filtered.png', bbox_inches='tight')

# write
q_atchafalaya_godin.dropna().to_csv(
    delft3d_input / f'{site_name}_{begin_date}_godin.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

## Wax Lake Outlet

In [ ]:
# download
site_name = "wax_lake_outlet_at_calumet"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_wax_lake = download_discharge_nwis(site_name, site_no, begin_date, end_date)
q_wax_lake_godin = apply_godin(q_wax_lake)

# visualize
fig, axes = plt.subplots(nrows=2, figsize=(16, 9))
q_wax_lake.plot(ax=axes[0])
q_wax_lake_godin.plot(ax=axes[0])
axes[0].set_ylabel('discharge [$m^3/s$]')
axes[0].set_title(site_name)

q_wax_lake.plot(ax=axes[1])
q_wax_lake_godin.plot(ax=axes[1])
axes[1].set_xlim(pd.to_datetime('2018-02-07'), pd.to_datetime('2018-04-29'))
axes[1].set_ylabel('discharge [$m^3/s$]')
fig.savefig(figures / f'{site_name}_filtered.png', bbox_inches='tight')

# write
q_wax_lake_godin.dropna().to_csv(
    delft3d_input / f'{site_name}_{begin_date}_godin.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

# Belle Chasse

In [ ]:
# download and tidally filter Belle Chasse data
site_name = "mississippi_at_belle_chase"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_mississippi = download_discharge_nwis(site_name, site_no, begin_date, end_date)
q_mississippi_godin = apply_godin(q_mississippi)

# visualize
fig, axes = plt.subplots(nrows=2, figsize=(16, 9))
q_mississippi.plot(ax=axes[0])
q_mississippi_godin.plot(ax=axes[0])
axes[0].set_ylabel("discharge [$m^3/s$]")
axes[0].set_title("Belle Chasse")

q_mississippi.plot(ax=axes[1])
q_mississippi_godin.plot(ax=axes[1])
axes[1].set_xlim(pd.to_datetime("2018-06-23"), pd.to_datetime("2018-07-4"))
axes[1].set_ylim(8000, 18000)
axes[1].set_ylabel("discharge [$m^3/s$]")
fig.savefig(figures / "belle_chasse_filtered.png", bbox_inches="tight")

q_mississippi_godin.dropna().to_csv(
    delft3d_input / f"{site_name}_{begin_date}_godin.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Blind River (New canal river)

In [ ]:
# visualize
site_name = "new_canal_river_east_sorrento"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_new_canal = download_discharge_nwis(site_name, site_no, begin_date, end_date)

q_new_canal_godin = apply_godin(q_new_canal)

fig, axes = plt.subplots(nrows=2, figsize=(16, 9))
q_new_canal.plot(ax=axes[0])
q_new_canal_godin.plot(ax=axes[0])
axes[0].set_ylabel("discharge [$m^3/s$]")
axes[0].set_title(site_name)

q_new_canal.plot(ax=axes[1])
q_new_canal_godin.plot(ax=axes[1])
axes[1].set_xlim(pd.to_datetime("2018-06-23"), pd.to_datetime("2018-07-4"))
axes[1].set_ylim(-20, 40)
axes[1].set_ylabel("discharge [$m^3/s$]")
fig.savefig(figures / f"blind_river-{site_name}_filtered.png", bbox_inches="tight")

q_new_canal_godin.dropna().to_csv(
    delft3d_input / f"blind_river-{site_name}_{begin_date}_godin.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Amite River

In [ ]:
# visualize
site_name = 'amite_river_at_port_vincent'
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_amite = download_discharge_nwis(site_name, site_no, begin_date, end_date)
q_amite_godin = apply_godin(q_amite)

fig, axes = plt.subplots(nrows=2, figsize=(16, 9))
q_amite.plot(ax=axes[0])
q_amite_godin.plot(ax=axes[0])
axes[0].set_ylabel('discharge [$m^3/s$]')
axes[0].set_title(site_name)

q_amite.plot(ax=axes[1])
q_amite_godin.plot(ax=axes[1])
axes[1].set_xlim(pd.to_datetime('2018-06-23'), pd.to_datetime('2018-07-4'))
axes[1].set_ylim(-10, 100)
axes[1].set_ylabel('discharge [$m^3/s$]')
fig.savefig(figures / f'{site_name}_filtered.png', bbox_inches='tight')

q_amite_godin.dropna().to_csv(
    delft3d_input / f'{site_name}_{begin_date}_godin.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

## The Tickfaw and Natalbany Rivers (Tickfaw source)

In [ ]:
# The Tickfaw doesn't needs to be tidally filtered
site_name = "tickfaw_river_at_holden"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])
q_tickfaw = download_discharge_nwis(site_name, site_no, begin_date, end_date)

# The Natalbany River doesn't need to be tidally filtered
site_name2 = "natalbany_river_at_baptist"
site_no = sites[site_name2][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])
q_natalbany = download_discharge_nwis(site_name, site_no, begin_date, end_date)

# there are duplicates in the Tickfaw data
q_tickfaw_natalbany = combine_discharges([q_tickfaw, q_natalbany])

fig, axes = plt.subplots(nrows=3, figsize=(16, 12), sharex=True, sharey=True)

q_tickfaw.plot(ax=axes[0])
axes[0].set_title(site_name)
axes[0].set_ylabel("discharge [$m^3/s$]")
axes[0].set_xlabel("")

q_natalbany.plot(ax=axes[1])
axes[1].set_title(site_name2)
axes[1].set_ylabel("discharge2 [$m^3/s$]")
axes[1].set_xlabel("")


q_tickfaw.plot(ax=axes[2], label="dis1")
q_natalbany.plot(ax=axes[2])
q_tickfaw_natalbany.plot(ax=axes[2])
axes[2].set_title(site_name + " and " + site_name2)
axes[2].set_ylabel("discharge [$m^3/s$]")
axes[2].set_xlabel("")
axes[2].legend(["Tickfaw", "Natalbany", "Combined"])

fig.savefig(figures / f"{site_name}_and_{site_name2}_filtered.png", bbox_inches="tight")

q_tickfaw_natalbany.to_csv(
    delft3d_input / f"Tickfaw_River_{site_name}_and_{site_name2}_{begin_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Tangipahoa River

In [ ]:
# visualize
# no need to filter
site_name = 'tangipahoa_river_at_robert'
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_tangipahoa = download_discharge_nwis(site_name, site_no, begin_date, end_date)
q_tangipahoa_godin = apply_godin(q_tangipahoa)

fig, axes = plt.subplots(nrows=2, figsize=(16, 9))
q_tangipahoa.plot(ax=axes[0])
q_tangipahoa_godin.plot(ax=axes[0])
axes[0].set_ylabel('discharge [$m^3/s$]')
axes[0].set_title(site_name)

q_tangipahoa.plot(ax=axes[1])
q_tangipahoa_godin.plot(ax=axes[1])
axes[1].set_xlim(pd.to_datetime('2018-06-23'), pd.to_datetime('2018-07-4'))
axes[1].set_ylim(-10, 100)
axes[1].set_ylabel('discharge [$m^3/s$]')
fig.savefig(figures / f'{site_name}_{begin_date}_filtered.png', bbox_inches='tight')

q_tangipahoa.dropna().to_csv(
    delft3d_input / f'{site_name}_{begin_date}.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

## The Tchefuncte River

In [ ]:
# visualize
site_name = "tchefuncte_near_folsom"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_tchefuncte = download_discharge_nwis(site_name, site_no, begin_date, end_date)

fig, ax = plt.subplots(figsize=(16, 4))
q_tchefuncte.plot(ax=ax)
ax.set_ylabel("discharge [$m^3/s$]")
ax.set_title(site_name)

fig.savefig(figures / f"{site_name}_{begin_date}.png", bbox_inches="tight")

q_tchefuncte.dropna().to_csv(
    delft3d_input / f"{site_name}_{begin_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Jourdan River

In [ ]:
# visualize
site_name = "jourdan_river"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])


# The mean over calendar days is taken from 2000-10-10 to 2004-09-30 after being Godin filtered
q_jourdan = download_discharge_nwis(site_name, site_no, "2000-10-10", "2004-09-30", skiprows=31)
q_jourdan = q_jourdan.where(q_jourdan.discharge > -50, np.nan).dropna()
q_jourdan_godin = apply_godin(q_jourdan)

q_jourdan_godin["dayofyear"] = q_jourdan_godin.index.dayofyear
q_jourdan_godin_mean = q_jourdan_godin.groupby(by=["dayofyear"]).mean().dropna()
time_idx = pd.date_range("2018-01-01", "2019-01-01", freq="1D")
q_jourdan_godin_mean = pd.Series(q_jourdan_godin_mean.discharge_cms_Godin_filtered.values, index=time_idx)

fig, ax = plt.subplots(figsize=(16, 4))
q_jourdan_godin_mean.plot(ax=ax)
ax.set_ylabel('discharge [$m^3/s$]')
ax.set_title(site_name)
fig.savefig(figures / f'{site_name}_{begin_date}-{end_date}_daily_means_godin.png', bbox_inches='tight')

q_jourdan_godin_mean.dropna().to_csv(
    delft3d_input / f"{site_name}_{begin_date}-{end_date}_daily_means_godin.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Wolf River

In [ ]:
# visualize
site_name = 'wolf_river'
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_wolf_river = download_discharge_nwis(site_name, site_no, begin_date, end_date)

fig, ax = plt.subplots(figsize=(16, 4))
q_wolf_river.plot(ax=ax)
ax.set_ylabel('discharge [$m^3/s$]')
ax.set_title(site_name)
fig.savefig(figures / f'{site_name}_{begin_date}-{end_date}.png', bbox_inches='tight')

q_wolf_river.dropna().to_csv(
    delft3d_input / f'{site_name}_{begin_date}-{end_date}.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

## Biloxi

In [ ]:
# visualize
# no filtering needed
site_name = "biloxi_river"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_biloxi_river = download_discharge_nwis(site_name, site_no, begin_date, end_date)

# add historical daily mean from Tuxachanie creek
site_name2 = "tuxachanie_creek"
site_no2 = sites[site_name2][0]
site_info.loc[int(site_no2), ["used"]] = True
print(site_info.loc[int(site_no2)])
q_tuxachanie = pd.read_csv(
    input_data / f"{site_name2}_historical_mean_daily_cfs.txt",
    sep="\s+",
    skiprows=46,
    usecols=[4, 5, 13],
    names=["month", "day", "discharge"],
)

q_tuxachanie.discharge = q_tuxachanie.discharge * cfs2cms

# create datetime index and remove leap day for 2018
timestamps = [
    f"2018-{month}-{day}" for month, day in zip(q_tuxachanie.month, q_tuxachanie.day)
]
q_tuxachanie = pd.DataFrame(
    data=q_tuxachanie.discharge.values,
    index=pd.to_datetime(timestamps),
    columns=["discharge"],
    dtype=float,
)

q_biloxi_tuxachanie = combine_discharges([q_biloxi_river, q_tuxachanie])

# visualize
fig, axes = plt.subplots(nrows=3, figsize=(16, 12))
q_biloxi_river.plot(ax=axes[0])
q_tuxachanie.plot(ax=axes[1])

q_biloxi_tuxachanie.plot(ax=axes[2])
q_biloxi_river.plot(ax=axes[2])
q_tuxachanie.plot(ax=axes[2])

axes[0].set_ylabel("discharge [$m^3/s$]")
axes[0].set_title(site_name)
axes[1].set_ylabel("discharge [$m^3/s$]")
axes[1].set_title("Tuxachanie Creek daily mean")
axes[2].set_ylabel("discharge [$m^3/s$]")
axes[2].legend(["combined", "Biloxi River", "Tuxachanie"])

fig.savefig(
    figures / f"Biloxi_bay_{site_name}_and{site_name2}_{begin_date}-{end_date}.png",
    bbox_inches="tight",
)

# write out data
q_biloxi_tuxachanie.dropna().to_csv(
    delft3d_input / f"Biloxi_Bay_{site_name}_and_{site_name2}_{begin_date}-{end_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## The Pearl River

In [ ]:
site_name = "bogue_chitto_river"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])
q_bogue_chitto = download_discharge_nwis(site_name, site_no, begin_date, end_date)

site_name2 = "pearl_at_bogalusa"
site_no2 = sites[site_name2][0]
site_info.loc[int(site_no2), ["used"]] = True
print(site_info.loc[int(site_no2)])
q_pearl_river = download_discharge_nwis(site_name2, site_no2, begin_date, end_date)

site_name3 = "west_hobolochitto"
site_no3 = sites[site_name3][0]
site_info.loc[int(site_no3), ["used"]] = True
print(site_info.loc[int(site_no3)])
q_west_hobo = download_discharge_nwis(site_name3, site_no3, begin_date, end_date)

site_name4 = "east_hobolochitto"
site_no4 = sites[site_name4][0]
site_info.loc[int(site_no4), ["used"]] = True
print(site_info.loc[int(site_no4)])
q_east_hobo = download_discharge_nwis(site_name4, site_no4, begin_date, end_date)

q_combined_pearl_river = combine_discharges(
    [q_bogue_chitto, q_pearl_river, q_west_hobo, q_east_hobo]
)

# Visualize and write combined data
fig, axes = plt.subplots(nrows=5, figsize=(16, 18), sharex=True)

q_combined_pearl_river.plot(ax=axes[-1])
axes[-1].set_title("Pearl River (combined discharges)")
axes[-1].set_ylabel("discharge [$m^3/s$]")

q_bogue_chitto.plot(ax=axes[0])
axes[0].set_title(site_name)
axes[0].set_ylabel("discharge [$m^3/s$]")
q_bogue_chitto.plot(ax=axes[-1])

q_pearl_river.plot(ax=axes[1])
axes[1].set_title(site_name2)
axes[1].set_ylabel("discharge [$m^3/s$]")
q_pearl_river.plot(ax=axes[-1])

q_west_hobo.plot(ax=axes[2])
axes[2].set_title(site_name3)
axes[2].set_ylabel("discharge [$m^3/s$]")
q_west_hobo.plot(ax=axes[-1])

q_east_hobo.plot(ax=axes[3])
axes[3].set_title(site_name4)
axes[3].set_ylabel("discharge [$m^3/s$]")
q_east_hobo.plot(ax=axes[-1])

axes[-1].legend(["combined", site_name, site_name2, site_name3, site_name4])

q_combined_pearl_river.dropna().to_csv(
    delft3d_input / f"Pearl_River_combined_stations_{begin_date}-{end_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Pascagoula River

In [ ]:
# Red Creek is a tributary of the Pascagoula.
# The Pascagoula River has data between 1993 - 2009, so daily mean's will be used initially.
# Escatawpa at I-10 has discharge data form 2001-08-26 to 2004-09-30.
# Compare I-10 obs (larger catchment) to Escatawpa at Agricola and scale the data by the mean of their ratio

begin_date_for_ratio = "2001-08-26"
end_date_for_ratio = "2003-09-30"
q_escatawpa_I10_for_ratio = (
    download_discharge_nwis(
        "escatawpa_at_I-10", "0248018020", begin_date_for_ratio, end_date_for_ratio, skiprows=29
    )
    .groupby(pd.Grouper(freq="1D"))
    .mean()
)

q_escatawpa_agricola_for_ratio = (
    download_mean_discharge_nwis(
        "escatawpa_at_agricola", "02479560", begin_date_for_ratio, end_date_for_ratio
    )
    .groupby(pd.Grouper(freq="1D"))
    .mean()
)

ratio = (q_escatawpa_I10_for_ratio / q_escatawpa_agricola_for_ratio).mean()

site_name = "escatawpa_river_nr_agricola"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])
q_escatawpa = download_mean_discharge_nwis(site_name, site_no, begin_date, end_date)
q_escatawpa_scaled = q_escatawpa * ratio

# add historical daily mean for Pascagoula creek
site_name2 = "pascagoula_river"
site_no2 = sites[site_name2][0]
site_info.loc[int(site_no2), ["used"]] = True
print(site_info.loc[int(site_no2)])

q_pascagoula = pd.read_csv(
    input_data / f"{site_name2}_historical_mean_daily.txt",
    sep="\s+",
    skiprows=46,
    usecols=[4, 5, 13],
    names=["month", "day", "discharge"],
)

q_pascagoula.discharge = q_pascagoula.discharge * cfs2cms
timestamps = [
    f"2018-{month}-{day}" for month, day in zip(q_pascagoula.month, q_pascagoula.day)
]
q_pascagoula = pd.DataFrame(
    data=q_pascagoula.discharge.values,
    index=pd.to_datetime(timestamps),
    columns=["discharge"],
    dtype=float,
)

# combine
q_escatawpa_pascagoula = combine_discharges([q_escatawpa_scaled, q_pascagoula])

# visualize
fig, axes = plt.subplots(nrows=3, figsize=(16, 12))
q_escatawpa_scaled.plot(ax=axes[0])
q_escatawpa.plot(ax=axes[0])
axes[0].legend(["scaled", "observed"])
q_pascagoula.plot(ax=axes[1])

q_escatawpa_pascagoula.plot(ax=axes[2])
q_escatawpa_scaled.plot(ax=axes[2])
q_pascagoula.plot(ax=axes[2])

axes[0].set_ylabel("discharge [$m^3/s$]")
axes[0].set_title("Escatawpa River")
axes[1].set_ylabel("discharge [$m^3/s$]")
axes[1].set_title("The Pascagoula River")
axes[2].set_ylabel("discharge [$m^3/s$]")
axes[2].legend(["combined", "Escatawpa", "Pascagoula"])

fig.savefig(
    figures / f"Pascagoula_{site_name}_and_{site_name2}_{begin_date}-{end_date}.png",
    bbox_inches="tight",
)

# write out data
(q_escatawpa_pascagoula / 2).dropna().to_csv(
    delft3d_input
    / f"Pascagoula_{site_name}_and_{site_name2}_{begin_date}-{end_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

# write out data
(q_escatawpa_pascagoula / 2).dropna().to_csv(
    delft3d_input
    / f"West_Pascagoula_{site_name}_and_{site_name2}_{begin_date}-{end_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

## Mobile Bay

### Head of the Bay

In [ ]:
# From inspection of the NHD for Alabama it appears that the Tombigbee and Alabama Rivers are tributaries of the Mobile River.
# The Mobile River then branches which are both gages (Mobile river and Tensaw)
# The Chickasaw joins wtih the Mobile River down stream of the gage
# combine The Chickasaw with the Mobile for output, keep Tensaw separate

# mobile river and tensaw have tidally filtered data available
data_code = 72137
site_name = "mobile_river_at_bucks"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_mobile = download_discharge_nwis(
    site_name, site_no, begin_date, end_date, data_code
)

site_name2 = "tensaw_at_mount_vernon"
site_no2 = sites[site_name2][0]
site_info.loc[int(site_no2), ["used"]] = True
print(site_info.loc[int(site_no2)])
q_tensaw = download_discharge_nwis(
    site_name2, site_no2, begin_date, end_date, data_code, 29
)

site_name3 = "chickasaw_creek"
site_no3 = sites[site_name3][0]
site_info.loc[int(site_no3), ["used"]] = True
print(site_info.loc[int(site_no3)])
q_chickasaw = download_mean_discharge_nwis(site_name3, site_no3, begin_date, end_date)

# combine
# Remove duplicates
q_mobile_chickasaw = combine_discharges([q_mobile, q_chickasaw])

## Visualize
fig, axes = plt.subplots(nrows=3, figsize=(16, 8), sharex=True, sharey=True)
ax1 = axes[0]
ax2 = axes[1]
ax3 = axes[2]
q_tensaw.plot(ax=ax1)
ax1.set_title("tensaw_at_mount_vernon")
ax1.set_ylabel("discharge [$m^3/s$]")
ax1.grid()

q_chickasaw.plot(ax=ax2)
ax2.set_title("Chickasaw creek")
ax2.set_ylabel("discharge [$m^3/s$]")

q_mobile.plot(ax=ax3)
q_mobile_chickasaw.plot(ax=ax3, ls='--')
ax3.set_title("mobile_river_at_bucks")
ax3.set_ylabel("discharge [$m^3/s$]")
ax3.legend(["Mobile River", "Combined w/ Chickasaw"])
ax3.grid()

fig.savefig(
    figures / "mobile_and_tensaw_2018_tidally_filtered.png", bbox_inches="tight"
)

# write out data
q_mobile_chickasaw.dropna().to_csv(
    delft3d_input / f'Mobile_{begin_date}_combined_w_Chickasaw_Creek.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

# write out data
q_tensaw.dropna().to_csv(
    delft3d_input / f'tensaw_at_mount_vernon_{begin_date}.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

### Mouth of the bay

#### Fowl River

In [ ]:
# visualize
# doesn't need to be filtered
site_name = "fowl_river"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_fowl_river = download_discharge_nwis(site_name, site_no, begin_date, end_date)

fig, ax = plt.subplots(figsize=(16, 5))
q_fowl_river.plot(ax=ax)
ax.set_ylabel('discharge [$m^3/s$]')
ax.set_title(site_name)
fig.savefig(figures / f'{site_name}_{begin_date}.png', bbox_inches='tight')

q_fowl_river.dropna().to_csv(
    delft3d_input / f'{site_name}_{begin_date}.csv',
    sep='\t',
    header=['discharge_m3s'],
    index_label=['datetime_UTC']
)

#### Magnolia River and Fish Creek

In [ ]:
# visualize
# doesn't need to be filtered
fig, axes = plt.subplots(nrows=3, sharex=True, sharey=True, figsize=(16, 9))

site_name = "fish_river"
site_no = sites[site_name][0]
site_info.loc[int(site_no), ["used"]] = True
print(site_info.loc[int(site_no)])

q_fish_river = download_discharge_nwis(site_name, site_no, begin_date, end_date)


site_name2 = "magnolia_river"
site_no2 = sites[site_name2][0]
site_info.loc[int(site_no2), ["used"]] = True
print(site_info.loc[int(site_no2)])

q_magnolia_river = download_discharge_nwis(site_name, site_no, begin_date, end_date)

q_fish_river.plot(ax=axes[0])
axes[0].set_title(site_name)
axes[0].set_ylabel("discharge [$m^3/s$]")

q_magnolia_river.plot(ax=axes[1])
axes[1].set_ylabel("discharge [$m^3/s$]")
axes[1].set_title(site_name2)

q_magnolia_fish_rivers = combine_discharges([q_magnolia_river, q_fish_river])
q_magnolia_fish_rivers.plot(ax=axes[2])
q_fish_river.plot(ax=axes[2])
q_magnolia_river.plot(ax=axes[2])
axes[2].set_ylabel("discharge [$m^3/s$]")
axes[2].set_title("Combined")
axes[2].legend(["Combined", "Fish", "Magnolia"])

fig.savefig(
    figures / f"Combined_Magnolia_Fish_Rivers_{begin_date}-{end_date}.png", bbox_inches="tight"
)

q_magnolia_fish_rivers.dropna().to_csv(
    delft3d_input / f"combined_magnolia_fish_rivers_{begin_date}.csv",
    sep="\t",
    header=["discharge_m3s"],
    index_label=["datetime_UTC"],
)

# Updated site info

In [ ]:
used_site_list_fn = site_info_output / "used_usgs_gages.csv"
site_info.drop("geometry", axis=1)[site_info.used == True].to_csv(used_site_list_fn)

site_info_fn = site_info_output / "usgs_gages.shp"
site_info.to_file(site_info_fn)
site_info[site_info.used == True]

# Nice visualization

In [ ]:
# visualize nice
fig, ax = plt.subplots(figsize=(12, 5))
q_mississippi.plot(ax=ax)
q_mississippi_godin.plot(ax=ax)
ax.set_ylabel("discharge [$m^3/s$]")
ax.set_title("Belle Chasse")
ax.legend(["USGS observations", "tidally filtered"])
fig.savefig(figures / "belle_chasse_2018.png", bbox_inches="tight")

In [ ]:
# visualize nice
fig, ax = plt.subplots(figsize=(12, 5))

q_combined_pearl_river.plot(ax=ax)
q_pearl_river.plot(ax=ax)
q_bogue_chitto.plot(ax=ax)
q_west_hobo.plot(ax=ax)
q_east_hobo.plot(ax=ax)

ax.set_ylabel("discharge [$m^3/s$]")
ax.set_title("Pearl River")
ax.legend(
    [
        "Discharge input",
        "Pearl River",
        "Bogue Chitto",
        "West Hobolochitto",
        "East Hobolochitto",
    ],
    ncol=2
)
fig.savefig(figures / "pearl_river_2018.png", bbox_inches="tight")

In [ ]:
# visualize nice
fig, ax = plt.subplots(figsize=(12, 5))

q_mobile.plot(ax=ax)
q_tensaw.plot(ax=ax)

ax.set_ylabel("discharge [$m^3/s$]")
ax.set_title("Mobile/Tensaw River")
ax.legend(
    [
        "Mobile River",
        "Tensaw River",
    ]
)
fig.savefig(figures / "mobile_river_2018.png", bbox_inches="tight")